## Backpropagation for humans


This is probably the least understood algorithm in Machine Learning but is extremely intuitive. In this post we'll explore how to mathematically derive backpropagation and get an intuition how it works.

### How does a Neural Network learn? 
The learning process is simply adjusting the weights and biases that's it! The Neural Netowork does this by a process called Backpropagation. The steps are as follows:
1. Randomly initialise weights
2. __Forward Pass__: Predict a value using an activation function. 
3 See how bad you're performing using loss function. 
4. __Backward Pass__: Backpropagate the error. That is, tell your network that it's wrong, and also tell what direction it's supposed to go in order to reduce the error. This step updates the weights (here's where the network learns!)
5. Repeat steps 2 & 3 until the error is reasonably small or for a specified number of iterations. 

Step 3 is the most important step. We'll mathematically derive the equation for updating the values. 

### Activation function 
Here's a wonderful post I wrote that covers activation functions in detail [link]. 


### Loss function
Loss function is used to check how bad you're network is performing. One simple way to do that is subtract the predicted value and the actual value. For instance, if the actual value is 45 and your network is predicting 30, you can extrapolate you network is off by 15. 

In practise we don't use the simple subtraction, we instead square. Why do we square it? Well the square function has better mathematical properties (like it's a convex function) which are beyond the scope of this blog. While it's possible to use any loss function, we'll use mean squared error for now to keep things simple which is given by

$$ \tag 1
J\left(\theta\right)=\sum_{i=0}^m\left(y^{\left(i\right)} - h_{\theta}\left(x^{\left(i\right)}\right)\right)^2
$$

Which is simply the sum of the squared difference between the obeserved value and predicted value. 

In the above equation
1. m = number of examples
2. $h(\theta)$ = activation function
3. $x^{(i)}$ = the ith sample in the dataset
4. $y^{(i)}$ = output of ith sample in the dataset

We'll be using sigmoid activation function which is simplest to illustrate while deriving Backpropagation. Refer [link] for the derivation of the gradient of sigmoid which is given by

$$ \tag 2
\frac{\partial}{\partial x}\sigma\left(x\right)=\sigma\left(x\right)\cdot\left(1-\sigma\left(x\right)\right)
$$

We'll denote the above equation by $\sigma'\left(x\right)$


## Vectorizing things
Instead of using nasty for loops which is not only messy but also susceptible to bugs, we will vectorize the forward and backward passses and let numpy take care of them. 

### Weighted sum

We will define our first layer weights as:
$$
\theta_1 = 
\begin{bmatrix}
\theta^{(1)}_{10} & \theta^{(1)}_{11} & \theta^{(1)}_{12} \\
\theta^{(1)}_{20} & \theta^{(1)}_{21} & \theta^{(1)}_{32} \\
\theta^{(1)}_{30} & \theta^{(1)}_{31} & \theta^{(1)}_{33} \\
\end{bmatrix}
$$

And second layer weights as:
$$
\theta_2 = 
\begin{bmatrix}
\theta^{(2)}_{10} & \theta^{(2)}_{11} & \theta^{(2)}_{12} & \theta^{(2)}_{13} \\
\end{bmatrix}
$$


As as result, instead of writing the weighted sum of second layer as:
$$ z_1^{\left(2\right)}=\theta_{10}^{\left(1\right)}+\theta_{11}^{\left(1\right)}x_1+\theta_{12}^{\left(1\right)}x_2 + ....\text{for all the $z$s}$$

All we do is:
$$ \tag 3 z^{\left(2\right)}=\theta^{\left(2\right)}\cdot X^T $$

And the activity at the second layer is thus
$$ \tag 4 a^{\left(2\right)}=\sigma\left(z^{\left(2\right)}\right) $$
Which is the same as:
$$ \tag 5 a^{\left(2\right)}=\sigma\left(\theta^{\left(2\right)}\cdot X^T\right) $$

Repeating the same step for the third layer will give us the output. 
$$ \tag 6 z^{\left(3\right)}=\theta^{\left(2\right)}\cdot a^{\left(2\right)} $$
$$ \tag 7 a^{\left(3\right)}=\sigma\left(z^{\left(3\right)}\right) $$
$$ \tag 8 h_\theta(x^{(i)}) = a^{\left(3\right)}$$

## Forward Pass

Let's take an example of a Neural Network to solve the MNIST character recognition problem. Every image is 20x20 pixel in dimension, hence the a single input will (20x20) 400 features. Remember, that the input is the first layer, so the number of neurons in the first layer will be 400. The second layer will be the hidden layer, let's say that the number of neurons in the hidden layers is 25. And since we're predicting whether the image is a number from 0-9 there are 10 discrete outputs, hence the output layer will have 10 neurons. Each of the neuron in output layer will predict a value between 0 and 1. Since these values are probabilities, the value that has the highest probability will be the winner. 

#### Dimension of (input) X = (5000, 400) 

Then we'll append a column  of `1`s so that we will not have to multiply the bais term separately. Adding $1$ will multiply the bais term with $1$ and keep it unaltered.Since $a^{(1)}$ is nothing but input, appending a columns of $1$ will make it (5000, 401)

#### Dimension of $a^{(1)}$ = (5000, 401)

Now, to calculate, $z^{(2)}$ we see $\theta^{(1)}$ is a matrix like:
$$
\theta_1 = 
\begin{bmatrix}
\theta^{(1)}_{10} & \theta^{(1)}_{11} & \theta^{(1)}_{12} &...\\
\theta^{(1)}_{20} & \theta^{(1)}_{21} & \theta^{(1)}_{32} &...\\
\theta^{(1)}_{30} & \theta^{(1)}_{31} & \theta^{(1)}_{33} & ...\\
... & ... & ... & ... \\
\end{bmatrix}
$$

Whose dimension will be (number of hidden layers, number of input layers + 1). Where the +1 is the bias term. Note that a generalised dimension for $\theta$ in any layer will be (number of neurons in next layer, number of neurons in current layer + 1). 

#### Hence, dimension of $\theta^{(1)}$ will be (25, 401) 

And $a^{(1)}$ is a matrix like: 

$$
a^{(1)} = 
\begin{bmatrix}
1 & X^{(1)}_{1} & X^{(1)}_{2} & ...\\
1 & X^{(2)}_{1} & X^{(2)}_{2} & ...\\
1 & X^{(3)}_{1} & X^{(3)}_{2} & ...\\
... & ... & ... & ...
\end{bmatrix}
$$

But for $z^{(2)}$ the required operation is: 
$$z^{(2)}_{1} = \theta^{(1)}_{10} + \theta^{(1)}_{11} \cdot X^{(1)}_{1} + \theta^{(1)}_{12} \cdot X^{(1)}_{2}$$

Hence we'll have to transpose $a^{(2)}$ to make it work. So, $z^{(2)}$ now becomes
$$z^{(2)} = \theta^{(1)} \cdot a^{(1)T}$$

The dimension of multiplication can be expressed by: (25, 401) x (401, 5000) = (25, 5000)

#### Hence, dimension of $z^{(2)}$ will be (25, 5000)

As a practise, we'll always keep $a^{(i)}$s in the dimension of input. So $a^{(i)}$s will be always (5000, x) so that we can write generalized code. Hence while calculating $z^{(3)}$ we'll transpose it so that we have the shape of $z^{(3)}$ as (5000, x). Also note that we'll need to add a column $1$s for bias. 
$$a^{(2)} = \sigma(z^{(2)T})$$

#### After adding a column of $1$ dimensions of $a^{(2)}$ will be (5000, 26)

#### Also dimensions of $\theta^{(2)}$ will be (10, 26) 

Calculate $z^{(3)}$ in the same way as it's done above. 

$$z^{(3)} = \theta^{(2)} \cdot a^{(2)T}$$

Similarly, the dimension of multiplication can be expressed by: (10, 26) x (26, 5000) = (10, 5000)

#### Hence dimensions of $z^{(3)}$ will be (10, 5000)

Since this is the last (output) layer, there's no need to add an addition column of $1$ like we did while calculating the output $a^{(2)}$

$$a^{(3)} = \sigma(z^{(3)T})$$

#### Dimensions of $a^{(3)}$ will be (5000, 10)

Hence, in the end we will have 10 predictions for every sample of data. 

## Backward pass

Inorder to derive the update rule, we need to know two equations:
### 1. $\frac{\partial J}{\partial \theta^{\left(1\right)}}$
This calculates the error with respect to the weights in first layer. What it does is asks the following question: "What direction should I change the weights in first layer, so the cost of the network is minimised?"

### 2. $\frac{\partial J}{\partial \theta^{\left(2\right)}}$
This calculates the error with respect to the weights in second layer. What it does is asks the following question: "What direction should I change the weights in second layer, so the cost of the network is minimised?"

## Easier part $\frac{\partial J}{\partial \theta^{\left(2\right)}}$

Calculating $\frac{\partial J}{\partial \theta^{\left(2\right)}}$ is easier than calculating $\frac{\partial J}{\partial \theta^{\left(1\right)}}$ so we'll start by that first. We'll go step by step and try to understand what each step is accomplishing. 



$$\begin{align}
\tag {from (1)}
\frac{\partial J}{\partial \theta^{\left(2\right)}} &= \frac{\partial\frac{1}{2}\sum_{i=0}^m\left(y-h_{\theta}(x)\right)^2}{\partial \theta^{\left(2\right)}} \\
\notag
&=\frac{\sum_{i=0}^m\partial\frac{1}{2}\left(y-h_{\theta}(x)\right)^2}{\partial \theta^{\left(2\right)}} \\
\end{align}$$


The summation sign simply sums up across all the examples. We'll skip the summation sign for now to make it look neat. We'll come back to it after some time. 

$$ \begin{align}
\notag
\frac{\partial J}{\partial \theta^{\left(2\right)}} &= \frac{\partial\frac{1}{2}\left(y-h_{\theta}(x)\right)^2}{\partial \theta^{\left(2\right)}} \\
\notag
&= (y-h_{\theta}(x))\cdot\left(-\frac{\partial h_{\theta}(x)}{\partial \theta^{\left(2\right)}}\right)
\end{align} 
$$

We have to differentiate $h_{\theta}(x)$ to respect the [Chain Rule](https://www.youtube.com/watch?v=6kScLENCXLg). This minus sign in the second term comes from differentiating $-h_{\theta}(x)$

Using Equation (7) and (8) we have, 

$$
\frac{\partial J}{\partial \theta^{\left(2\right)}} = (h_{\theta}(x)-y)\cdot \frac{\partial\sigma\left(z^{\left(3\right)}\right)}{\partial \theta^{\left(2\right)}}
$$

Notice we've taken the minus sign out and changed the signs of $y$ and $h_{\theta}(x)$. Now, we know what's the gradient of $\sigma$ using equation (2) so we'll simply substitue that. Futhermore, we'll apply chain rule to obtain the following. 
$$
\begin{align}
\notag
\frac{\partial J}{\partial \theta^{\left(2\right)}} &= \left(h_{\theta}(x)-y\right)\cdot\sigma'\left(z^{\left(3\right)}\right)\cdot\left(\frac{\partial z^{\left(3\right)}}{\partial \theta^{\left(2\right)}}\right) \\
\tag{Using (6)}
&= \left(h_{\theta}(x)-y\right)\cdot\sigma'\left(z^{\left(3\right)}\right)\cdot\left(\frac{\partial \left(\theta^{\left(2\right)} \cdot a^{\left(2\right)}\right)}{\partial \theta^{\left(2\right)}}\right)
\end{align}
$$

In the last part of the equation we'll be differentiating $\theta^{\left(2\right)} \cdot a^{\left(2\right)}$ by $\theta^{\left(2\right)}$. We know that the derivative of $4x$ with respect to $x$ is $4$ so the derivative of $\theta^{\left(2\right)} \cdot a^{\left(2\right)}$  with respect $\theta^{\left(2\right)}$ will be $a^{\left(2\right)}$

$$ 
\frac{\partial J}{\partial \theta^{\left(2\right)}} = \left(h_{\theta}(x)-y\right)\cdot\sigma'\left(z^{\left(3\right)}\right)\cdot\left(a^{\left(2\right)}\right)
$$

We'll denote the error term in the final layer by $\delta^{(3)}$

$$ 
\tag{9}
\frac{\partial J}{\partial \theta^{\left(2\right)}} = \delta^{\left(3\right)}\cdot a^{\left(2\right)}
$$

Now, coming back to the summation we ignored at the top of the derivation, we're going to fix that in the implementation using an accumulator matrix which will store the errors for every row and sum it up. 

## Sucky part $\frac{\partial J}{\partial \theta^{\left(1\right)}}$

It's nearly the same as the previous step, but involves one additional step using chain rule. We'll start in the same way. 

$
\begin{align}
\tag {from (1)}
\frac{\partial J}{\partial W^{\left(1\right)}} &= \frac{\partial\frac{1}{2}\sum_{i=0}^m\left(y-\hat y\right)^2}{\partial W^{\left(1\right)}} \\
\notag
&=\frac{\sum_{i=0}^m\partial\frac{1}{2}\left(y-z\right)^2}{\partial W^{\left(1\right)}} \\
&= (y-\hat y)\cdot\left(-\frac{\partial \hat y}{\partial W^{\left(1\right)}}\right)
\end{align}
$
Not that we've skipped the summation sign as before. Mathematicians might be cursing me at this point. 

$$
\begin{align}
\notag
\frac{\partial J}{\partial W^{\left(1\right)}} &= \left(z-y\right)\cdot\sigma'\left(z^{\left(3\right)}\right)\cdot\left(\frac{\partial z^{\left(3\right)}}{\partial W^{\left(1\right)}}\right) \\
\end{align}
$$

Things start to get a little different here. We cannot directly differentiate $z^{(3)}$ with respect to $W^{(1)}$ because $z^{(3)}$ does not directly depend on $W^{(1)}$. So we will use our good ol' chain rule again and divide it further.

$$
\frac{\partial J}{\partial W^{\left(1\right)}} = \left(z-y\right)\cdot\sigma'\left(z^{\left(3\right)}\right)\cdot \frac{\partial z^{\left(3\right)}}{\partial a^{\left(2\right)}}\cdot\frac{\partial a^{\left(2\right)}}{\partial W^{\left(1\right)}}
$$

Replacing the value of $\delta^{(3)}$ from equation (9)

$$
\frac{\partial J}{\partial W^{\left(1\right)}} = \delta^{(3)} \cdot \frac{\partial z^{\left(3\right)}}{\partial a^{\left(2\right)}}\cdot\frac{\partial a^{\left(2\right)}}{\partial W^{\left(1\right)}}
$$

Substituting the value of $z^{(3)}$ from equation (6)

$$
\begin{align}
\notag
\frac{\partial J}{\partial W^{\left(1\right)}} &= \delta^{(3)} \cdot \frac{\partial z^{\left(3\right)}}{\partial a^{\left(2\right)}}\cdot\frac{\partial a^{\left(2\right)}}{\partial W^{\left(1\right)}} \\
\notag
&= \delta^{(3)} \cdot \frac{\partial\left(W^{\left(2\right)}\cdot a^{\left(2\right)}\right)}{\partial a^{\left(2\right)}} \cdot\frac{\partial a^{\left(2\right)}}{\partial W^{\left(1\right)}} \\
\notag
&= \delta^{(3)} \cdot W^{(2)} \cdot \frac{\partial a^{\left(2\right)}}{\partial W^{\left(1\right)}} \\
\tag{Using (4)}
&= \delta^{(3)} \cdot W^{(2)} \cdot \frac{\partial\sigma\left(z^{\left(2\right)}\right)}{\partial W^{\left(1\right)}} \\
\tag{We've done this before}
&= \delta^{(3)} \cdot W^{(2)} \cdot \sigma'\left(z^{\left(2\right)}\right) \cdot \frac{\partial z^{\left(2\right)}}{\partial W^{\left(1\right)}} \\
\tag{Using (3)} 
&= \delta^{(3)} \cdot W^{(2)} \cdot \sigma'\left(z^{\left(2\right)}\right) \cdot \frac{\partial\left(W^{\left(1\right)}\cdot a^{\left(1\right)}\right)}{\partial W^{\left(1\right)}} \\
\notag
&= \delta^{(3)} \cdot W^{(2)} \cdot \sigma'\left(z^{\left(2\right)}\right) \cdot a^{\left(1\right)}
\end{align}
$$

Like before, we'll use $\delta^{(2)}$ in place of the error.
$$\delta^{\left(2\right)} = \delta^{\left(3\right)}\cdot W^{\left(2\right)}\cdot\sigma'\left(z^{\left(2\right)}\right)$$

Hence, we have the final equation,

$$
\tag {10}
\frac{\partial J}{\partial W^{\left(1\right)}} = \delta^{(2)} \cdot a^{(1)}$$

Note the striking similarity between equation (9) and (10) ? That's the magic! This is how you can just stack a bunch of these operations and create a deep neural network with ease. 